In [ ]:
import numpy as np
import pandas as pd

pd.options.display.float_format='{:.2f}'.format

df = pd.read_csv('../data/000015', index_col= 'Date', names=['Date', 'Open', 'Close', 'High', 'Low', 'Volume', 'Money', 'PE', 'PB'], parse_dates=True, header=None)
df['Return'] = df['Close'].pct_change() * 100
df['Year'] = [i.year for i in df.index]
df['Month'] = [i.month for i in df.index]
df['Day'] = [i.day for i in df.index]

# Basic indexing

In [ ]:
df_prices = df[['Close', 'High', 'Low']]
   
series_close = df['Close']
close_of_a_day = series_close['2010-01-04']
close_of_a_day

# can't get row of dataframe like:
try:
    df_prices['2010-01-04']
except KeyError:
    pass

## Accessing attributes using dot operator

In [ ]:
df.Close

# select by specifying column indexes
df[[1, 2]]

## Range slicing

The synttax of the slicing operator exactly matches that of NumPy:

```python
ar[startIndex: endIndex: stepValue]
```

where the default values if not specified are as follows:

* 0 for startIndex
* arraysize-1 for endIndex
* 1 for stepValue

# Label, integer and mixed indexing

* The `.loc` operator: Allows label-oriented indexing
* The `.iloc` operator: Allows integer-based indexing
* The `.ix` operator: Allows mixed label and integer-based indexing


## Label-oriented indexing

The `.loc` operator supports pure label-based indexing. It accepts the following as valid inputs:

* A single label.
* List or array of labels.
* A slice object with labels.
* A Boolean array.

In [ ]:
df.loc['2010-01-04']

# follows are same
df.loc['2010-01-04', 'Close']
df.loc['2010-01-04']['Close']
df['Close']['2010-01-04']

df.loc[['2010-01-04', '2010-01-05']]
df.loc['2010-01-04': '2010-02-05']

### Selection using a Boolean array

In [ ]:
df.loc[df['Close'] <= df['Close'].min(),:]

## Integer-oriented indexing

The `iloc` operator supports integer-based positional indexing. It accepts the following as inputs:

* A single integer.
* A list or array of integers.
* A slice object with integers.

In [ ]:
df.iloc[0:10,]

## Mixed indexing with the .ix opeator

The `.ix` operator behaves like a mixture of the `.loc` and `.iloc` operators, with the `.loc` behavior taking precedence. It takes the following as possible inputs:

* A single label or integer
* A list of integers or labels
* An integer slice or label slice
* A Boolean array

In [ ]:
df.ix['2010-01-04']
df.ix[['2010-01-04', '2010-01-05']]
df.ix[df.index[-3:]]
df.ix[0]
df.ix[[0, 2]]
df.ix[1: 3]
df.ix[df['Close'] > 4044.6640]

## MultiIndexing

In [ ]:
df1 = df.reset_index()
df1.set_index(['Year', 'Month', 'Day'], inplace=True)

df1.index.get_level_values(0)
df1.index.get_level_values(1)
df1.index.get_level_values(2)

df1.ix[2011, 2]
df1.ix[2011: 2012]
df1.ix[(2011, 1): (2012,2)]

## Swapping and reordering levels

In [ ]:
df_swapped = df1.swaplevel(0, 1, axis=0)
df_swapped.sortlevel(0).ix[(1,2010):(1,2011)]

# recorder_levels function is more general

df_recorded = df1.reorder_levels(['Month', 'Day', 'Year'], axis = 0)

## Cross sections

The `xs` method provides a shortcut means of selecting data based on a particular index level value.

In [ ]:
df1.xs(2, level='Month')

# same as 

df1.swaplevel(0, 1, axis=0).ix[2]

# Boolean indexing

## isin and any all methods

In [ ]:
df.loc[df.index.isin(['2010-01-04', '2010-01-05'])]
df.loc[(df.astype('int') == 2656).any(axis = 1)]

## using the where() method

The `where` method is used to ensure that the result of Boolean filtering is the same shape as the original data.

In [ ]:
df[df['Close'] > 2800]
df.where(df > 2800)

## Operations on indexes

In [ ]:
df1 = df.reset_index()
df1.set_index('Date')

# Grouping of data

## The groupby operation

The `groupby` operation can be thought of as part of a process that involves the following three steps:

* Splitting the dataset
* Analyzing the data
* Aggregating or combining the data

The result of a `groupby` operation is not a DataFrame but `dict` of DataFrame objects.

In [ ]:
df['Year'] = [i.year for i in df.index]
df['Month'] = [i.month for i in df.index]
df['Day'] = [i.day for i in df.index]

df_group_by_year = df.groupby('Year')
type(df_group_by_year)
len(df_group_by_year)
df_group_by_year.size().sort_values(ascending=False)

df_group_by_year_month = df.groupby(['Year', 'Month'])
df_group_by_year_month.size().sort_values(ascending=False)

df_group_by_year = df.groupby(lambda x: x.year)
#for name, group in df_group_by_year:
#    print(name)
#    print(group)

df_index_year_month = df.reset_index()
df_index_year_month = df_index_year_month.set_index(['Year', 'Month'])
df_group_by_year = df_index_year_month.groupby(level=['Year', 'Month'])
df_group_by_year.size()

## Using groupby with a MultiIndex

In [31]:
df_index_year_month = df.reset_index()
df_index_year_month = df_index_year_month.set_index(['Year', 'Month'])

grouped = df_index_year_month.groupby(level='Month')
grouped.mean()
# same as
df_index_year_month.mean(level='Month')

grouped.sum()

,Open,Close,High,Low,Volume,Money,PE,PB,Return,Day
Month,,,,,,,,,,
1,344277.66,344149.13,347879.65,340574.16,399257643400.00,3215136303977.00,251.95,1958.41,-18.67,2536
2,289976.39,290403.18,292439.04,287702.28,271977747500.00,2182408625726.00,211.35,1667.02,13.44,2001
3,365081.84,365537.72,368104.22,362244.03,433170024600.00,3519677097840.00,272.89,2127.33,14.58,2518
4,318079.23,318643.96,320900.02,315429.01,464051855400.00,4024152390707.00,228.97,1860.74,13.02,2208
5,327718.30,327819.79,330657.64,324464.30,340975920400.00,3083895206775.00,234.83,1905.39,-10.87,2458
6,311967.23,311790.39,314684.48,307726.84,410889258200.00,3655267164643.00,216.12,1731.28,-34.70,2168
7,325469.67,325989.29,329494.87,321890.94,457179547900.00,3708001449623.00,232.61,1895.07,-2.93,2470
8,324774.43,325102.74,328113.28,321774.93,277546303700.00,2170428631852.00,227.66,1836.83,-11.80,2472
9,286273.51,286456.67,288746.69,283903.64,223138600900.00,1685074595013.00,201.16,1634.10,-3.02,2181


## Using the aggregate method

Another way to generate summary statistics by using the aggregate method explicitly:

In [ ]:
grouped.aggregate(np.sum)

## Applying multiple functions

In [12]:
grouped.agg([np.sum, np.mean, np.size])

grouped['Return'].agg({'Size': np.size, 'Total': np.sum, 'Average': np.mean, 'Deviation': np.std, 'Max': np.max, 'Min': np.min})

,Size,Total,Average,Deviation,Max,Min
Month,,,,,,
1,154.00,-18.67,-0.12,1.91,4.20,-8.59
2,131.00,13.44,0.10,1.37,3.65,-6.16
3,163.00,14.58,0.09,1.36,4.66,-4.93
4,137.00,13.02,0.10,1.26,3.82,-5.03
5,146.00,-10.87,-0.07,1.52,3.39,-7.09
6,138.00,-34.70,-0.25,1.80,6.23,-7.62
7,155.00,-2.93,-0.02,1.92,6.87,-8.42
8,155.00,-11.80,-0.08,1.81,6.31,-9.22
9,140.00,-3.02,-0.02,1.39,5.53,-3.90


## The transform() method

The `groupby-transform` function is used to perform transormation operation on a `group` object. For example, we could replace NaN values in the `groupby` object using the `fillna` method. The resulting object after using the transform has the same size as the original `groupby` object.

In [52]:
grouped.transform(lambda x: x.fillna(x.mean()))

Open   Close    High     Low        Volume          Money   PE  \
Year Month                                                                     
2010 1     2827.01 2777.19 2831.87 2776.92 1254719900.00 12684101201.00 2.78   
     1     2784.76 2814.05 2818.82 2758.52 1565152100.00 17214700158.00 2.81   
     1     2807.63 2796.78 2841.60 2795.72 1419338100.00 16095862114.00 2.78   
     1     2794.45 2726.52 2810.70 2718.57 1462168100.00 15453928317.00 2.70   
     1     2716.81 2730.01 2734.99 2692.33 1078873800.00 11860712296.00 2.68   
     1     2841.88 2737.01 2841.88 2725.16 1792579200.00 17535349849.00 2.75   
     1     2730.65 2779.87 2781.02 2703.05 1808328300.00 19659493369.00 2.79   
     1     2713.19 2676.16 2734.96 2670.34 2362136400.00 23905525318.00 2.68   
     1     2689.67 2710.16 2714.05 2668.39 1472738700.00 15952317361.00 2.70   
     1     2710.09 2715.04 2735.23 2688.01 1310731400.00 13999464170.00 2.71   
     1     2707.74 2729.33 2731.17 2698.80 1498092500.00 15424612993.00 2.71   
     1     2733.35 2735.00 2760.60 2730.89 1467177500.00 15683881214.00 2.71   
     1     2742.11 2644.80 2746.60 2644.30 1789303000.00 17871589767.00 2.62   
     1     2650.29 2658.09 2672.58 2632.55 1437154400.00 14721781636.00 2.63   
     1     2624.89 2651.05 2667.64 2578.88 2000508000.00 18456551200.00 2.63   
     1     2622.56 2603.15 2655.96 2602.21 1073696400.00  9500090593.00 2.60   
     1     2603.13 2550.54 2614.57 2530.94 1209330800.00 11297571033.00 2.55   
     1     2550.46 2510.91 2558.02 2498.68 1096387400.00  9847856320.00 2.52   
     1     2508.93 2501.31 2520.34 2488.42  916576000.00  8610553487.00 2.51   
     1     2491.59 2494.00 2525.20 2480.30  904982300.00  8760461387.00 2.50   
     2     2488.16 2452.03 2489.29 2428.59 1009153100.00  9631897379.00 2.45   
     2     2466.06 2449.50 2496.61 2447.96  862002200.00  8206056620.00 2.45   
     2     2463.53 2517.09 2519.02 2420.55 1341765600.00 12469436544.00 2.52   
     2     2499.91 2502.13 2522.59 2488.05  997133800.00  9635897340.00 2.50   
     2     2452.20 2452.15 2469.41 2436.70 1106442000.00 10152580605.00 2.45   
     2     2450.25 2448.40 2464.03 2437.62  616202100.00  5920995163.00 2.45   
     2     2447.83 2465.33 2472.66 2444.07  852792900.00  7260659909.00 2.45   
     2     2482.02 2489.75 2490.93 2471.05  666177600.00  6318344754.00 2.48   
     2     2490.73 2494.76 2506.64 2483.32  724020400.00  6770275817.00 2.48   
     2     2504.76 2524.93 2525.71 2501.92  675488100.00  6382508232.00 2.51   
...            ...     ...     ...     ...           ...            ...  ...   
2017 1     2649.45 2647.87 2660.51 2642.25 1657828900.00 13595831749.00 1.24   
     1     2646.78 2661.94 2666.46 2645.48 1779837200.00 13866257203.00 1.36   
     1     2659.85 2668.25 2669.01 2654.68 1367166800.00 11438640886.00 1.25   
     1     2668.92 2670.18 2679.79 2666.68 1512966400.00 11630026971.00 1.36   
     2     2670.47 2649.09 2672.02 2647.02 1035209900.00  8741841165.00 1.36   
     2     2653.15 2656.27 2663.04 2646.30 1576381000.00 13072956938.00 1.24   
     2     2654.19 2657.90 2661.91 2648.78 1387776900.00 11672337444.00 1.24   
     2     2654.29 2664.12 2664.72 2641.80 1437285400.00 11928433379.00 1.25   
     2     2664.11 2674.00 2681.76 2662.39 2101850300.00 18006006129.00 1.25   
     2     2674.75 2691.68 2694.18 2673.94 2772966200.00 24267670531.00 1.38   
     2     2692.70 2699.80 2702.77 2691.68 2525171300.00 22577109788.00 1.27   
     2     2699.26 2693.60 2701.97 2686.32 1692077300.00 15214082035.00 1.26   
     2     2694.38 2703.58 2722.29 2692.29 3822563200.00 27215620989.00 1.28   
     2     2703.69 2713.38 2713.66 2700.06 2430063300.00 18550864342.00 1.28   
     2     2711.77 2692.27 2713.54 2690.56 2060320700.00 16677134422.00 1.26   
     2     2691.60 2732.25 2733.26 2691.60 3743705200.00 29073613195.00 1.27   
     2     2734.31 2734.98 2748.43 2731.32 2545240000.00 20930422505.00 1.28   
     2     273

## Filtering

The `filter` method enables to apply filtering on a `groupby` object that results in a subset of the initial object.

In [48]:
grouped.filter(lambda x: np.all([x[col] > 1640 for col in ['High', 'Close', 'Low']]))

Date    Open   Close    High     Low        Volume  \
Year Month                                                            
2010 10    2010-10-08 2117.28 2168.04 2174.39 2115.32 1289840500.00   
     10    2010-10-11 2188.37 2230.67 2241.52 2187.85 2099275100.00   
     10    2010-10-12 2222.97 2250.15 2256.02 2211.16 1863045700.00   
     10    2010-10-13 2255.10 2275.28 2276.05 2245.72 1808492300.00   
     10    2010-10-14 2288.79 2298.89 2326.51 2283.18 2519840300.00   
     10    2010-10-15 2290.66 2392.98 2393.31 2285.72 2912805900.00   
     10    2010-10-18 2404.73 2389.46 2451.55 2386.93 3148225800.00   
     10    2010-10-19 2388.82 2429.14 2429.14 2366.67 2035218500.00   
     10    2010-10-20 2386.68 2426.51 2453.81 2379.60 2477544800.00   
     10    2010-10-21 2433.49 2403.12 2441.88 2384.57 1833918000.00   
     10    2010-10-22 2396.52 2400.56 2416.43 2385.67 1440724100.00   
     10    2010-10-25 2402.99 2449.89 2450.97 2381.21 1839875600.00   
     10    2010-10-26 2458.13 2433.82 2468.49 2418.27 1831273000.00   
     10    2010-10-27 2428.84 2418.42 2455.84 2411.61 1882167600.00   
     10    2010-10-28 2410.55 2417.03 2429.70 2397.66 1571432300.00   
     10    2010-10-29 2417.14 2381.80 2417.26 2372.14 1499275900.00   
2011 10    2011-10-10 1899.47 1885.61 1902.78 1881.38  434627000.00   
     10    2011-10-11 1934.90 1880.59 1944.20 1865.18  877086100.00   
     10    2011-10-12 1867.78 1934.89 1936.59 1855.82  996683000.00   
     10    2011-10-13 1926.13 1941.66 1944.91 1920.67  880295200.00   
     10    2011-10-14 1935.28 1938.61 1943.36 1920.56  543153300.00   
     10    2011-10-17 1940.11 1942.50 1954.84 1931.37  524167400.00   
     10    2011-10-18 1929.37 1896.44 1933.24 1893.31  647713400.00   
     10    2011-10-19 1901.37 1896.28 1915.08 1891.96  469971100.00   
     10    2011-10-20 1885.89 1860.34 1891.47 1849.49  602213200.00   
     10    2011-10-21 1862.15 1855.66 1873.20 1854.63  435874600.00   
     10    2011-10-24 1861.17 1902.14 1905.28 1856.96  633015900.00   
     10    2011-10-25 1898.77 1930.35 1937.72 1891.85  773442600.00   
     10    2011-10-26 1922.00 1942.75 1960.38 1919.98  880132600.00   
     10    2011-10-27 1945.48 1948.89 1962.54 1940.41  724262000.00   
...               ...     ...     ...     ...     ...           ...   
2015 10    2015-10-13 2600.73 2617.93 2620.92 2587.52 2286774000.00   
     10    2015-10-14 2604.82 2581.36 2622.91 2581.10 2109040300.00   
     10    2015-10-15 2581.26 2645.43 2645.76 2580.15 2481737100.00   
     10    2015-10-16 2666.19 2685.26 2685.96 2650.50 3350608100.00   
     10    2015-10-19 2701.03 2696.63 2716.50 2671.41 3156726900.00   
     10    2015-10-20 2695.55 2712.12 2712.12 2675.23 2566278900.00   
     10    2015-10-21 2712.64 2635.47 2734.92 2597.20 6145816000.00   
     10    2015-10-22 2619.35 2669.40 2672.54 2614.16 3300780400.00   
     10    2015-10-23 2680.20 2708.03 2714.19 2668.87 2973537300.00   
     10    2015-10-26 2738.89 2720.34 2743.51 2702.80 3177125100.00   
     10    2015-10-27 2704.64 2721.19 2727.57 2649.40 2540243200.00   
     10    2015-10-28 2710.22 2695.27 2743.44 2690.05 2639813200.00   
     10    2015-10-29 2701.11 2702.76 2717.21 2687.31 1919541900.00   
     10    2015-10-30 2698.06 2690.46 2713.73 2673.71 2133711700.00   
2016 10    2016-10-10 2492.62 2516.48 2516.53 2486.57 1486931600.00   
     10    2016-10-11 2518.98 2524.27 2525.22 2514.66 1449201200.00   
     10    2016-10-12 2517.75 2515.00 2517.81 2505.29 1034843000.00   
     10    2016-10-13 2514.36 2524.89 2525.34 2512.08 1225308700.00   
     10    2016-10-14 2521.38 2525.27 2525.74 2510.53 1342116000.00   
     10    2016-10-17 2527.20 2508.36 2530.49 2502.63 1366481100.00   
     10    2016-10-18 2506.77 2543.69 2544.44 2506.02 1637575500.00   
     10    2016-10-19 2545.64 2543.25 2555.24 2537.11 1479974800.00   
     10    2016-10-20 2543.19 2544.13 2547.00 2535.11 1241371500.00   
     10    2016-10-21 2542.95 2559.47 256